### Objetivo do Nobook 
<p>O objetivo deste notebook é extrair o histórico de preços de ações de banco de dados relacional, acrescentar os indicadores técnicos e gravá-los no formato <b>parquet</b> para que sejam usados no processo de preparação dos dados para o treinamento do modelo de machine learning. O primeiro passo é obter os dados históricos de negociação de cada ativo gravá-los como arquivos individuais em disco. Estes arquivos devem conter as seguintes colunas:</p>
<ul>
<li>ticker - Código de negociação do ativo</li>
<li>dt_price - Data de negociação</li>
<li>open - Preço de abertura no dia</li>
<li>close - Preço de fechameto no dia</li>
<li>high - Preço máximo no dia</li>
<li>low - Preço mínimo no dia</li>
<li>volume - Volume financeiro negociado do ativo no dia</li>
</ul>
<p>É fundamental que os arquivos contenham exatamente estas colunas, pois a biblioteca TA-Lib (utilizada para calcularos indicadores técnicos) espera estes nomes como parâmetros de entrada para suas funções.</p>
<p><b>Exemplo</b>:</p>
<p>Lista de ativos recuperados do banco de dados: [PETR4, ITUB4, VALE3, ABEV3]</p>
<p>Arquivos gerados:</p>
<ul>
<li> PETR4.parquet </li>
<li> ITUB4.parquet </li>
<li> VALE3.parquet </li>
<li> ABEV3.parquet </li>
<ul>

In [1]:
# Carrega as variáveis de embiente usadas pelo notebook. Estas variáveis devem ser criadas no arquivo .env, localizada no mesmo diretório do notebook
%load_ext dotenv
%dotenv

In [ ]:
from data_preparation import PreProcess
from db_access import StockHistory, ExportToParquet
import os


# Obtem o historico de precos dos ativos 
str_conn = os.environ.get("STR_CON")
raw_fil_path = os.environ.get("RAW_DATA_PATH")
exporter = ExportToParquet()
dbo = StockHistory(str_conn)
df = dbo.select()

for stock in df['ticker'].unique():
    print(f"Processando ativo {stock}")
    df_aux = df.loc[df['ticker'] == stock]
    exporter.export(df_aux, raw_fil_path, stock)

<p>Uma vez que os arquivos com os dados históricos tenham sido gerados, o segudo passo da preparação dos datasets é iniciado. Neste passo serão incluídos indicadores técnicos ao histórico de preço de cada ativo para montar as estratégias de negociação que serão usados como entrada para os modelos. Uma estratégia de negociação nada mais é do que a combinação de indicadores técnicos que serão usados como features em modelos de machine learning. Durante o processo de treinamento serão usadas diversas combinações de indicadores e períodos diferentes com o objetivo de encontrar a configuração que obtenha a melhor acurácia na seleção dos ativos mais rentáveis.</p>
<p>Para automatizar o processo, as estratégias de negociação são configuradas em um arquivo json com a seguinte estrutura:</p>
<code>
<pre>
    "&lt;nome da estrategia&gt;":{
        "description": "&lt;Descrição da estratégia&gt;",
        "historic_period": &lt;Número inteiro que irá definir o total dias usados pelo modelo. Por exemplo, o modelo receberá um histórico de preço de 
                            60 dias para classificar o ativo&gt;,
        "profit_period": &lt;Número inteiro que definirá o período usado para calcular a rentabilidade do ativo (será usado para definir seu label).
                          Por exemplo, o modelo reberá 60 dias de cotação para classificar o ativo, se este parâmetro for setado para 10, a rentablidade 
                          usada para definir o label será calclada como: 
                          preço 10 dias após a o último dia da série de entrada/preço do último dia da série de entrada - 1&gt;,
        "profit": &lt;Rentabilidade usada para definir o label de classificação de cada registro enviado para o modelo durante o treinamento. Por exemplo,
                   Se o objetivo do modelo for identificar ativos que possuam rentabiliade superior a 5% em um período de 20 dias, o valor atribuído para
                   este parâmetro deve ser 0.05&gt;,
        "functions": { -- Lista de funções que fazem parte da estratégia de negociação
            "&lt;nome "fantasia" da função&gt;":{
                "function": "&lt;nome da função na biblioteca TA-Lib, será usado para chamar a função no código&gt;",
                "params":{ -- Lista de parâmetros da função (parâmetros relacionados ao preço do ativo não devem ser listados aqui)
                    "&lt;nome da parâmetro&gt;": &lt;valor&gt;7
                }                
            }
        }
    }
</pre>
</code>
<p>Exemplo de configuração:</p>
<p>Neste exemplo, a estratégia considera 90 dias de histórico de preço do ativo para classificar ativos que tenham rentabilidade superior a 5% em um período de 22 dias e para fazer isso, utiliza além do histórico de cotações os indicadores: Média móvel exponencial de 7 e 21 dias e o OBV.</p>
<code>
<pre>
{
    "EMA_7_21_OBV":{
        "description": "Combinacao de 2 medias moveis (7 e 21 dias) e OBV ",
        "historic_period": 90,
        "profit_period": 22,
        "profit": 0.05,
        "functions": {
            "EMA_7":{
                "function": "EMA",
                "params":{
                    "timeperiod": 7
                }
            },
            "EMA_21": {
                "function": "EMA",
                "params":{
                    "timeperiod": 21
                }            
            },
            "OBV":{
                "function": "OBV"                  
            }
        }
    }
}
</pre>
</code>